# Statistical Learning HW3: Classification models

In [2]:
%matplotlib inline
import pandas
import pickle
import numpy as np
# from scipy.stats import multivariate_normal

## Q1. Classification via Generative Models

We are going to explore the problem of identifying smartphone position through probabilistic generative models. Motion sensors in smartphones provide valuable information for researchers to understand its owners. An interesting (and more challenging) task is to identify human activities through the data recorded by motion sensors. For example, we want to know whether the smartphone owner is walking, running, or biking. In this homework problem, we are going to tackle a simpler problem. We want to know the static position of the smartphone. There are six possible positions:
 * Phoneonback: The phone is laying on the back of the phone with the screen pointing up (away from the ground).
 * Phoneonfront: The phone is laying on the back of the phone with the screen pointing towards the ground
 * Phoneonbottom: The phone is standing on the bottom of the screen, meaning the bottom is pointed towards the ground
 * Phoneontop: The phone is standing on the top of the screen, meaning the top is pointed towards the ground
 * Phoneonleft: The phone is laying on the left side of the screen.
 * Phoneonright: The phone is laying on the right side of the screen.

The input data is the reading of the accelerometer (cf. https://en.wikipedia.org/wiki/Accelerometer) in the smartphone. We have a training dataset that contains about 28,500 data points for phones in each of the six positions. The following is some basic information of the training dataset.

### Construct mypgc class

In [2]:
class mypgc:
        
    def fit(self, x_train, y_train):

        positions = y_train.unique()
        self.positions = positions

        data_dic = {}
        for po in self.positions:

            innder_dic = {}
            
            filter_data = x_train.loc[x_train['label'] == po]
            po_data = filter_data.drop(["label"], axis = 1).values

            #the mean vector
            mean = po_data.mean(axis=0)
            innder_dic["mu"] = mean

            #the covariance matrix
            cov = np.cov(po_data.T)
            innder_dic["cov"] = cov

            #the inverse of covariance matrix
            inverse = np.linalg.inv(cov)
            innder_dic["prec"] = inverse

            #logarithm of the determinant of the covariance matrix
            detcov = np.linalg.slogdet(cov)
            innder_dic["detcov"] = detcov

            #number of observations for this label
            n = len(po_data)
            innder_dic["n"] = n

            #learned prior probability of this label
            prior = n/len(y_train)
            innder_dic["prior"] = prior

            data_dic[po] = innder_dic
            self.trained_model = data_dic
        
    def predict(self, X_test):

        predictions = []
        for x in X_test:
            
            posteriors = []
            for p in self.positions:
                
                likelihood = multivariate_normal.pdf(x, mean = self.trained_model[p]["mu"],
                                                     cov = self.trained_model[p]["cov"])
                prior_p = self.trained_model[p]["prior"]
                posterior = likelihood * prior_p
                posteriors.append(posterior)
            
            max_index = posteriors.index(max(posteriors))
            pre = self.positions[max_index]
            predictions.append(pre)

        return predictions

### Load training data

In [3]:
with open('phone_train.pickle', 'rb') as fh1:
    traindata = pickle.load(fh1)
    
print("The first few rows are:")
traindata.head()
print("\nSummary statistics:")
print(traindata.describe())
print("\nLabel counts:")
print(traindata['label'].value_counts())

The first few rows are:

Summary statistics:
                   x              y              z
count  167097.000000  167097.000000  167097.000000
mean        0.357340       0.146807      -0.015550
std         5.622396       5.552010       5.737150
min        -9.770279      -9.966507      -9.908417
25%         0.016617      -0.074875      -0.221497
50%         0.142776       0.009628       0.025223
75%         0.249893       0.295715       0.151032
max        10.073685       9.980255      10.031113

Label counts:
Phoneonleft      29522
Phoneonfront     29079
Phoneonback      28566
Phoneonbottom    27842
Phoneontop       26401
Phoneonright     25687
Name: label, dtype: int64


In [4]:
Y_train = traindata['label']
X_train = traindata

### Load testing data

In [5]:
with open('phone_test1.pickle', 'rb') as fh2:
    test_data = pickle.load(fh2)

print("The first few rows are:")
print(test_data.head())
print("\nSummary statistics:")
print(test_data.describe())
print("\nLabel counts:")
print(test_data['label'].value_counts())

The first few rows are:
          x         y         z          label
0 -0.213196  0.031982 -9.370026   Phoneonfront
1 -0.151306  9.761749  0.188354  Phoneonbottom
2 -0.122742  9.771271  0.277618  Phoneonbottom
3 -0.200104  9.805786  0.235962  Phoneonbottom
4 -0.244141  0.039124 -9.412872   Phoneonfront

Summary statistics:
                  x             y             z
count  83511.000000  83511.000000  83511.000000
mean       0.283334      0.133812      0.363750
std        5.612133      5.551263      5.703844
min       -9.979858     -9.890595     -9.534271
25%       -0.217957     -0.053711      0.193115
50%        0.168472      0.049835      0.305191
75%        0.375565      0.220779      0.584885
max       10.068436      9.888657     10.371780

Label counts:
Phoneonleft      14779
Phoneonfront     14517
Phoneonback      14306
Phoneonbottom    13887
Phoneontop       13183
Phoneonright     12839
Name: label, dtype: int64


In [6]:
Y_test = test_data['label']
X_test = test_data.drop(["label"], axis = 1).values

### Initiate mypgc object with data and make prediction

In [7]:
pgc1 = mypgc()
pgc1.fit(X_train, Y_train)
test_1 = X_test
ypred = pgc1.predict(test_1)

In [8]:
correctness = 0

print("prediction/correct label")
print("------------------------")
for i in range(len(Y_test)):
    
    if i < 20:
        print(ypred[i], "/", Y_test[i])
    if ypred[i] == Y_test[i]:
        correctness += 1
accuracy = correctness/len(Y_test)
print("------------------------")
print("accuracy: ", accuracy)

prediction/correct label
------------------------
Phoneonfront / Phoneonfront
Phoneonbottom / Phoneonbottom
Phoneonbottom / Phoneonbottom
Phoneonbottom / Phoneonbottom
Phoneonfront / Phoneonfront
Phoneonright / Phoneonright
Phoneonfront / Phoneonfront
Phoneontop / Phoneontop
Phoneonfront / Phoneonfront
Phoneonfront / Phoneonfront
Phoneonfront / Phoneonfront
Phoneonback / Phoneonback
Phoneonleft / Phoneonleft
Phoneonback / Phoneonback
Phoneonbottom / Phoneonbottom
Phoneonback / Phoneonback
Phoneonright / Phoneonright
Phoneonright / Phoneonright
Phoneontop / Phoneontop
Phoneonleft / Phoneonleft
------------------------
accuracy:  1.0


# Q2. Logistic Regression with L2 Regularization

We are going to use to "Adult" dataset on the UCI machine learning reposition https://archive.ics.uci.edu/ml/datasets/Adult. The goal is to predict the label values of the income column, which can be either '>50K' or '<=50K.' The dataset had splitted the training and test data, and we are going to respect this particular train-test split in model testing.

### Q2.1 Data preparation and data cleaning

In [3]:
# Load all datasets
train = pandas.read_csv("adult_train.csv") 
test = pandas.read_csv("adult_test.csv")

### data preprocessing

In [11]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler

#fix the extra "." in test data.
test['income'].replace(regex=True,inplace=True,to_replace=r'\.',value=r'')

num_col = set(train.describe().columns)
cat_col = set(train.columns) - num_col

#want to have order
num_col = list(num_col)
cat_col = list(cat_col)

print("There are %d categorical columns" % len(cat_col), ":", cat_col)
print("There are %d numerical columns" % len(num_col), ":", num_col)

show_details = 0
if show_details > 0:
    print("Check NA for numerical columns in training and test")
    print("... train")
    print(train[num_col].isna().any())
    print("... test")
    print(test[num_col].isna().any())

    print("Check NA for categorical columns")
    print("... train")
    print(train[cat_col].isna().any())
    print("... test")
    print(test[cat_col].isna().any())

    #there is no NA, but still need to handle the "?" problem.
    for acol in cat_col:
        print("---Unique values in ", acol)
        print(train[acol].value_counts())

#remove all rows with "?"
for acol in cat_col:
    print("--- Remove rows if have missing values in column", acol)
    ind1 = train[acol] != "?"
    train = train[ind1]

    ind2 = test[acol] != "?"
    test = test[ind2]

print("Data preprocessing completed.")

There are 9 categorical columns : ['workclass', 'race', 'income', 'education', 'sex', 'native-country', 'occupation', 'marital-status', 'relationship']
There are 6 numerical columns : ['capital-gain', 'hours-per-week', 'age', 'fnlwgt', 'capital-loss', 'education-num']
--- Remove rows if have missing values in column workclass
--- Remove rows if have missing values in column race
--- Remove rows if have missing values in column income
--- Remove rows if have missing values in column education
--- Remove rows if have missing values in column sex
--- Remove rows if have missing values in column native-country
--- Remove rows if have missing values in column occupation
--- Remove rows if have missing values in column marital-status
--- Remove rows if have missing values in column relationship
Data preprocessing completed.


### summary statistics

In [12]:
print("Training data summary statistics")
print(train.describe())
for acol in cat_col:
    print("---Value count for", acol)
    print(train[acol].value_counts())

print("\n\n Test data summary statistics")    
print(test.describe())
for acol in cat_col:
    print("---Value count for", acol)
    print(test[acol].value_counts())
    
    
print("Training data shape:", train.shape)
print("Test data shape:", test.shape)

Training data summary statistics
                age        fnlwgt  education-num  capital-gain  capital-loss  \
count  32561.000000  3.256100e+04   32561.000000  32561.000000  32561.000000   
mean      38.581647  1.897784e+05      10.080679   1077.648844     87.303830   
std       13.640433  1.055500e+05       2.572720   7385.292085    402.960219   
min       17.000000  1.228500e+04       1.000000      0.000000      0.000000   
25%       28.000000  1.178270e+05       9.000000      0.000000      0.000000   
50%       37.000000  1.783560e+05      10.000000      0.000000      0.000000   
75%       48.000000  2.370510e+05      12.000000      0.000000      0.000000   
max       90.000000  1.484705e+06      16.000000  99999.000000   4356.000000   

       hours-per-week  
count    32561.000000  
mean        40.437456  
std         12.347429  
min          1.000000  
25%         40.000000  
50%         40.000000  
75%         45.000000  
max         99.000000  
---Value count for workclass
 

In [14]:
train_y0 = train['income']
test_y0 = test['income']

train_x0 = train.loc[:, train.columns != 'income']
test_x0 = test.loc[:, test.columns != 'income']

yencoder = OneHotEncoder()
yencoder.fit(train_y0.values.reshape(-1, 1))

#the lable variables are ready
y_train = yencoder.transform(train_y0.values.reshape(-1, 1)).toarray()[:, 1]
y_test = yencoder.transform(test_y0.values.reshape(-1, 1)).toarray()[:, 1]


xstd = StandardScaler()
xstd.fit(train_x0[num_col].values.astype('float64'))
#keep a copy of feature names
feat_names = list(num_col.copy())
x_train = xstd.transform(train_x0[num_col].values.astype('float64'))
x_test = xstd.transform(test_x0[num_col].values.astype('float64'))

#remember to remove income
print("Doing one-hot encoding...")
for acol in set(cat_col) - set(['income']):
    print("dealing with column", acol)
    tmp1 = train_x0[acol].values.reshape(-1, 1)
    #establish code book using a threshold
    cc = train_x0[acol].value_counts()
    #frequency > 5
    #ind1 = cc > 5
    #codebook = cc[ind1].index.to_list()

    xenc = OneHotEncoder(sparse = False)
    xenc.fit(tmp1)
    thisnames = acol + "_" + xenc.categories_[0]
    xfeat_train = xenc.transform(tmp1)
    xfeat_test = xenc.transform(test_x0[acol].values.reshape(-1, 1))

    x_train = np.hstack((x_train, xfeat_train))
    x_test = np.hstack((x_test, xfeat_test))
    feat_names.extend(thisnames)

#check the minimal frequency requirement
min_freq = 10
col_count = np.sum(x_train, axis = 0)
ind_remove = col_count < min_freq
#do not remove numerical-valued columns
ind_remove[0:len(num_col)] = False

feat_names2 = np.array(feat_names)
print("Remove these columns:", feat_names2[ind_remove])

keepcol = np.logical_not(ind_remove)
x_train = x_train[:, keepcol]
x_test = x_test[:, keepcol]
feat_names_final = feat_names2[keepcol]

print("The shape of x_train", x_train.shape)
print("The shape of y_train", y_train.shape)
print("The shape of x_test", x_test.shape)
print("The shape of y_test", y_test.shape)

Doing one-hot encoding...
dealing with column workclass
dealing with column race
dealing with column education
dealing with column sex
dealing with column native-country
dealing with column occupation
dealing with column marital-status
dealing with column relationship
Remove these columns: ['workclass_ Never-worked' 'native-country_ Holand-Netherlands'
 'occupation_ Armed-Forces']
The shape of x_train (32561, 105)
The shape of y_train (32561,)
The shape of x_test (16281, 105)
The shape of y_test (16281,)


### Construct mylogistic_l2 class
+ Q2.2 Derive the gradient and hession matrix for the new E(w).
+ Q2.3 Create your own mylogistic_l2 class.

In [15]:
class mylogistic_l2:
    def __init__(self, reg_vec, max_iter, tol, add_intercept):
        """reg_vec: the regularization coefficient vector
           max_iter: maximum number of iteration to run for the Newton method
           tol: tolerance for the objective function
           add_intercept: whether to add intercept (a column of ones) at last column of the feature matrix"""
        self.reg_vec = reg_vec
        self.max_iter = max_iter
        self.tol = tol
        self.add_intercept = add_intercept
        
    def fit(self, x, y, verbal = False):
        if self.add_intercept:
            x = np.c_[x, np.ones(x.shape[0])]
        
        #initial value for w, using ridge regression solution
        inv_xtx = np.linalg.pinv(np.matmul(np.transpose(x), x) + np.diag(self.reg_vec))
        w = np.matmul(inv_xtx, np.matmul(np.transpose(x), y))

        obj_value = np.inf
        best_w = None
        tol = self.tol
        for iter in range(self.max_iter):
            if verbal: print("----Iteration", iter)
                
            y0 = 1 / (1 + np.exp(- np.matmul(x, w)))
            #gradient
            grad = self.reg_vec * w + np.matmul(np.transpose(x), y0 - y)
            R = np.diag(y0 * (1-y0))
            #hession
            hess = np.diag(self.reg_vec) + np.matmul(np.transpose(x), np.matmul(R, x))
            
            #newton update
            w = w - np.matmul(np.linalg.inv(hess), grad)
            y0 = 1 / (1 + np.exp(- np.matmul(x, w)))

            obj_value_new = np.sum(np.square(w) * self.reg_vec) * 0.5 - np.sum(y * np.log(y0) + (1-y) * np.log(1-y0))
            if verbal: 
                print("    obj_function value=", obj_value_new)
                print("    w[0:5] = ", w[0:5])
                print("    w[10:15] = ", w[10:15])
                print("    w[-1] = ", w[-1])
                
            if obj_value_new < obj_value:
                best_w = w

            if (obj_value - obj_value_new) < tol:
                self.niter = iter + 1
                break
            obj_value = obj_value_new
            
            self.best_w = best_w
            self.obj_value = obj_value

    def predict(self, x):
        """doing prediction"""
        if self.add_intercept:
            x = np.c_[x, np.ones(x.shape[0])]
        ypredprob = 1 / (1 + np.exp(- np.matmul(x, self.best_w)))
        ypred = (ypredprob >= 0.5).astype('float64')
        return(ypred)     

### Q2.3 Case 1: lambda = 1 for all coefficients¶

In [17]:
#### Case 1: 
lambda_vec = np.ones(x_train.shape[1] + 1) * 10
mlr2 = mylogistic_l2(reg_vec = lambda_vec, max_iter = 100, tol = 1e-5, add_intercept = True)
mlr2.fit(x_train, y_train, verbal = False)

print("num of iteration =", mlr2.niter)
print("obj_function value=", mlr2.obj_value)
print("w[0:5] (numerical-valued coef) = ", mlr2.best_w[0:5])
print("w[10:15] (binary-valued coef) = ", mlr2.best_w[10:15])
print("w[-1] (incercept) = ", mlr2.best_w[-1])

pred1 = mlr2.predict(x_test)

ncorrect = np.sum(y_test == pred1)
accuracy1 = ncorrect / y_test.shape[0]
print("accuracy for case 1:", accuracy1)

num of iteration = 7
obj_function value= 10433.611065257732
w[0:5] (numerical-valued coef) =  [2.19133161 0.36303875 0.34229569 0.07320921 0.25629539]
w[10:15] (binary-valued coef) =  [ 0.11734477 -0.53163682 -0.3299868  -0.20622546 -0.43847755]
w[-1] (incercept) =  -1.193220980731002
accuracy for case 1: 0.8530188563356059


### Q2.3 Case 2: lambda = 1 for all but the intercept, no regularization for incercept term.

In [20]:
#### Case 2
lambda_vec = np.ones(x_train.shape[1] + 1) * 10
lambda_vec[-1] = 0.0
mlr2 = mylogistic_l2(reg_vec = lambda_vec, max_iter = 100, tol = 1e-5, add_intercept = True)
mlr2.fit(x_train, y_train, verbal = False)

print("num of iteration =", mlr2.niter)
print("obj_function value=", mlr2.obj_value)
print("w[0:5] (numerical-valued coef) = ", mlr2.best_w[0:5])
print("w[10:15] (binary-valued coef) = ", mlr2.best_w[10:15])
print("w[-1] (incercept) = ", mlr2.best_w[-1])

pred2 = mlr2.predict(x_test)

ncorrect = np.sum(y_test == pred2)
accuracy2 = ncorrect / y_test.shape[0]
print("accuracy for case 2:", accuracy2)

num of iteration = 7
obj_function value= 10416.059842212042
w[0:5] (numerical-valued coef) =  [2.18855182 0.36394344 0.34518112 0.07351644 0.25647551]
w[10:15] (binary-valued coef) =  [ 0.2868292  -0.35565475 -0.16082062 -0.18409595 -0.24623374]
w[-1] (incercept) =  -2.942716176060639
accuracy for case 2: 0.8532031202014618


### Q2.3 Case 3: lambda = 1 for numerical-valued features, lambda = 0.5 for binary-valued features, no regularization for incercept term.

In [21]:
#### Case 3
lambda_vec = np.ones(x_train.shape[1] + 1) * 10
lambda_vec[-1] = 0.0
lambda_vec[len(num_col):] = 15
mlr2 = mylogistic_l2(reg_vec = lambda_vec, max_iter = 100, tol = 1e-5, add_intercept = True)
mlr2.fit(x_train, y_train, verbal = False)

print("num of iteration =", mlr2.niter)
print("obj_function value=", mlr2.obj_value)
print("w[0:5] (numerical-valued coef) = ", mlr2.best_w[0:5])
print("w[10:15] (binary-valued coef) = ", mlr2.best_w[10:15])
print("w[-1] (incercept) = ", mlr2.best_w[-1])

pred3 = mlr2.predict(x_test)

ncorrect = np.sum(y_test == pred3)
accuracy3 = ncorrect / y_test.shape[0]
print("accuracy for case 3:", accuracy3)

num of iteration = 7
obj_function value= 10469.90445697634
w[0:5] (numerical-valued coef) =  [2.18825202 0.36243945 0.3420522  0.07287258 0.25599129]
w[10:15] (binary-valued coef) =  [ 0.11133215 -0.53193164 -0.32000688 -0.14475314 -0.38754967]
w[-1] (incercept) =  -1.1469931181852007
accuracy for case 3: 0.8530188563356059


### Q2.4 Search for the best hyper-parameter

### Q2.5 Use sklearn.linear_model.LogisticRegression to train and test the model (including hyper-parameter tuning). Compare the estimated parameters and test accuracy with those from your own models.

In [19]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier.fit(adult_X_train, adult_Y_train)
x = classifier.predict(adult_X_test)
corrects = np.sum(x == adult_Y_test)
print("accuracy: ", corrects/len(adult_Y_test))

accuracy:  0.7924302788844622


In [23]:
# setting parameters
classifier = LogisticRegression(solver='newton-cg', max_iter = 1000, tol = 1e-5, fit_intercept = True)
classifier.fit(adult_X_train, adult_Y_train)
x = classifier.predict(adult_X_test)
corrects = np.sum(x == adult_Y_test)
print("accuracy: ", corrects/len(adult_Y_test))

accuracy:  0.847875166002656
